In [1]:
import tensorflow as tf
#import tensorflow_hub as hub
import requests
print(tf.__version__)
import zipfile
import requests
import os
import time
import pandas as pd
import random
import shutil
import tempfile
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger
import numpy as np
from PIL import Image
import pickle
from tensorflow.keras.models import load_model, Model
from PIL import Image
from PIL.PngImagePlugin import PngImageFile
import matplotlib.pyplot as plt
import glob
from functools import partial
import nltk
import absl

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except:
        print("Couldn't set memory_growth")
        pass
    
    
def fix_random_seed(seed):
    """ Setting the random seed of various libraries """
    try:
        np.random.seed(seed)
    except NameError:
        print("Warning: Numpy is not imported. Setting the seed for Numpy failed.")
    try:
        tf.random.set_seed(seed)
    except NameError:
        print("Warning: TensorFlow is not imported. Setting the seed for TensorFlow failed.")
    try:
        random.seed(seed)
    except NameError:
        print("Warning: random module is not imported. Setting the seed for random failed.")

# Fixing the random seed
random_seed=4321
fix_random_seed(random_seed)

2.4.1


http://archive.ics.uci.edu/ml/datasets/Forest+Fires

In [2]:
# Downloading the data
# http://www.thespermwhale.com/jaseweston/babi/CBTest.tgz

import os
import requests
import tarfile

import shutil

# Retrieve the data
if not os.path.exists(os.path.join('data', 'csv', 'forestfires.csv')):
    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv"
    # Get the file from web
    r = requests.get(url)

    if not os.path.exists(os.path.join('data', 'csv')):
        os.makedirs(os.path.join('data', 'csv'))
    
    # Write to a file
    with open(os.path.join('data', 'csv', 'forestfires.csv'), 'wb') as f:
        f.write(r.content)
        
else:
    print("The forestfires.csv file already exists.")
    
    
if not os.path.exists(os.path.join('data', 'forestfires.names')):
    
    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.names"
    # Get the file from web
    r = requests.get(url)

    if not os.path.exists('data'):
        os.makedirs('data')
    
    # Write to a file
    with open(os.path.join('data', 'forestfires.names'), 'wb') as f:
        f.write(r.content)
        
else:
    print("The forestfires.names file already exists.")



The forestfires.csv file already exists.
The forestfires.names file already exists.


In [3]:
# This is the directory containing the TFX Chicago Taxi Pipeline example.
_pipeline_root = os.path.join('pipeline', 'examples', 'forest_fires_pipeline')

# This is the path where your model will be pushed for serving.
# tempfile.mkdtemp() creates a temporary directory (e.g. tmpxxxxxx) in default OS temporary directory
_serving_model_dir = os.path.join(tempfile.mkdtemp(), 'serving_model', 'forest_fires')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)


In [4]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
# pipeline_name = "forest_fires", pipeline_root=_pipeline_root
context = InteractiveContext(pipeline_name = "forest_fires", pipeline_root=_pipeline_root)

INFO:absl:tensorflow_ranking is not available: No module named 'tensorflow_ranking'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'


## Had to install apache-beam 2.29 (not 2.30) coz 2.30 was giving the error
Attribute error: type StringMethods does not have attribute fullmatch

In [5]:
from tfx.components import CsvExampleGen
from tfx.proto import example_gen_pb2

output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=3),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))


example_gen = CsvExampleGen(input_base=os.path.join('data', 'csv'))
context.run(example_gen)

INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'
INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 63
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "pipeline\\examples\\forest_fires_pipeline\\CsvExampleGen\\examples\\1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:25478,xor_checksum:1623842899,sum_checksum:1623842899"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1623843296423
        last_update_time_since_epoch: 1623843298043
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [6]:
print(example_gen.outputs)

{'examples': Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 1
type_id: 5
uri: "pipeline\\examples\\forest_fires_pipeline\\CsvExampleGen\\examples\\1"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:25478,xor_checksum:1623842899,sum_checksum:1623842899"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    string_value: "0"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "0.29.0"
  }
}
state: LIVE
create_time_since_epoch: 1623843296423
last_update_time_since_epoch: 1623843298043
, artifact_type: id: 5
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value

In [7]:
!ls -lah /tmp/tfx-interactive-2021-06-11T07_06_53.060198-4jyvtunk/CsvExampleGen/examples/3
!ls -lah /tmp/tfx-interactive-2021-06-11T07_06_53.060198-4jyvtunk/CsvExampleGen/examples/3/Split-eval

'ls' is not recognized as an internal or external command,
operable program or batch file.
'ls' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)


["train", "eval"] pipeline\examples\forest_fires_pipeline\CsvExampleGen\examples\1


## Look at training examples

In [9]:
import pprint

pp = pprint.PrettyPrinter()

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(2):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)


features {
  feature {
    key: "DC"
    value {
      float_list {
        value: 669.0999755859375
      }
    }
  }
  feature {
    key: "DMC"
    value {
      float_list {
        value: 35.400001525878906
      }
    }
  }
  feature {
    key: "FFMC"
    value {
      float_list {
        value: 90.5999984741211
      }
    }
  }
  feature {
    key: "ISI"
    value {
      float_list {
        value: 6.699999809265137
      }
    }
  }
  feature {
    key: "RH"
    value {
      int64_list {
        value: 33
      }
    }
  }
  feature {
    key: "X"
    value {
      int64_list {
        value: 7
      }
    }
  }
  feature {
    key: "Y"
    value {
      int64_list {
        value: 4
      }
    }
  }
  feature {
    key: "area"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "day"
    value {
      bytes_list {
        value: "tue"
      }
    }
  }
  feature {
    key: "month"
    value {
      bytes_list {
        value: "oct"
    

## Generate schema

In [10]:
from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 64
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "pipeline\\examples\\forest_fires_pipeline\\StatisticsGen\\statistics\\2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1623843327385
        last_update_time_since_epoch: 1623843330314
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [11]:
context.show(statistics_gen.outputs['statistics'])

In [12]:
from tfx.components import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 65
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "pipeline\\examples\\forest_fires_pipeline\\SchemaGen\\schema\\3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1623843339632
        last_update_time_since_epoch: 1623843339675
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [13]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'day',STRING,required,single,'day'
'month',STRING,required,single,'month'
'DC',FLOAT,required,single,-
'DMC',FLOAT,required,single,-
'FFMC',FLOAT,required,single,-
'ISI',FLOAT,required,single,-
'RH',INT,required,single,-
'X',INT,required,single,-
'Y',INT,required,single,-


ValueError: Value must be a nonnegative integer or None

In [14]:
from tfx.components import ExampleValidator


example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)


INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 66
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 37
        type_id: 17
        uri: "pipeline\\examples\\forest_fires_pipeline\\ExampleValidator\\anomalies\\24"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1623879331625
        last_update_time_since_epoch: 1623879331686
        , artifact_type: id: 17
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [15]:
# When running
# context.show(example_validator.outputs['anomalies'])

# ValueError: Value must be a nonnegative integer or None
# pd.set_option('max_colwidth', -1)

https://www.tensorflow.org/tfx/tutorials/tfx/components

In [29]:
%%writefile forest_fires_constants.py

# Categorical features are assumed to each have a maximum value in the dataset.
MAX_CATEGORICAL_FEATURE_VALUES = [7, 12]

CATEGORICAL_FEATURE_KEYS = [
    'day', 'month'
]

DENSE_FLOAT_FEATURE_KEYS = ['DC', 'DMC', 'FFMC', 'ISI', 'rain', 'temp', 'wind', 'X', 'Y']

# Number of buckets used by tf.transform for encoding each feature.
BUCKET_FEATURE_BOUNDARIES = [(33, 66)]
BUCKET_FEATURE_KEYS = ['RH']

VOCAB_FEATURE_KEYS = ['day','month']
VOCAB_FEATURE_MAP_KEYS = [
    ('mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun'), 
    ('jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec')]
# Keys
LABEL_KEY = 'area'

def transformed_name(key):
  return key + '_xf'


Overwriting forest_fires_constants.py


In [17]:
%%writefile forest_fires_transform.py

import tensorflow as tf
import tensorflow_transform as tft

import forest_fires_constants

_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_BUCKET_FEATURE_BOUNDARIES = forest_fires_constants.BUCKET_FEATURE_BOUNDARIES
_LABEL_KEY = forest_fires_constants.LABEL_KEY
_transformed_name = forest_fires_constants.transformed_name


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _DENSE_FLOAT_FEATURE_KEYS:
    # Preserve this feature as a dense float, setting nan's to the mean.
    outputs[_transformed_name(key)] = tft.scale_to_z_score(
        sparse_to_dense(inputs[key])
    )

  # Treating categorical features  
  for key in _VOCAB_FEATURE_KEYS:
    # Build a vocabulary for this feature.
    outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
        sparse_to_dense(inputs[key]),
        num_oov_buckets=1)

    
  for key, boundary in zip(_BUCKET_FEATURE_KEYS, _BUCKET_FEATURE_BOUNDARIES):
    outputs[_transformed_name(key)] = tft.apply_buckets(
        sparse_to_dense(inputs[key]), bucket_boundaries=[boundary])


  outputs[_transformed_name(_LABEL_KEY)] = sparse_to_dense(inputs[_LABEL_KEY])

  return outputs

def sparse_to_dense(x):
    return tf.squeeze(
        tf.sparse.to_dense(tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1])),
        axis=1
    )


Overwriting forest_fires_transform.py


In [37]:
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath('forest_fires_transform.py'),
)

context.run(transform)


INFO:absl:Generating ephemeral wheel package for 'C:\\Users\\thush\\Documents\\code\\manning_tf2_in_action\\Ch14\\forest_fires_transform.py' (including modules: ['forest_fires_constants', 'forest_fires_trainer', 'forest_fires_transform']).
INFO:absl:User module package has hash fingerprint version 550e9c8bef5a8b7a4f10ba22afdead638f73c8e73acbc0c24f56159f76f0646c.
INFO:absl:Executing: ['c:\\anaconda3\\envs\\manning.tf2\\python.exe', 'C:\\Users\\thush\\AppData\\Local\\Temp\\tmpmk0hl3r3\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\thush\\AppData\\Local\\Temp\\tmp3afq116c', '--dist-dir', 'C:\\Users\\thush\\AppData\\Local\\Temp\\tmp2iga6w8x']
INFO:absl:Successfully built user code wheel distribution at 'pipeline\\examples\\forest_fires_pipeline\\_wheels\\tfx_user_code_Transform-0.0+550e9c8bef5a8b7a4f10ba22afdead638f73c8e73acbc0c24f56159f76f0646c-py3-none-any.whl'; target user module is 'forest_fires_transform'.
INFO:absl:Full user module path is 'forest_fires_transfor

ExecutionResult(
    component_id: Transform
    execution_id: 68
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 79
        type_id: 11
        uri: "pipeline\\examples\\forest_fires_pipeline\\Transform\\transform_graph\\67"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1624269509628
        last_update_time_since_epoch: 1624269542423
        , artifact_type: id: 11
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 80
        type_id: 5
        uri: "pipeline\\examples\\forest_fires_pipeline\\Transform\\transformed_examples\\67"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1624269509630
        last_update_time_since_epoch: 1624269542423
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 81
        type_id: 12
        uri: "pipeline\\examples\\forest_fires_pipeline\\Transform\\updated_analyzer_cache\\67"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1624269509630
        last_update_time_since_epoch: 1624269542423
        , artifact_type: id: 12
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [34]:
print(transform.outputs.keys())

dict_keys(['transform_graph', 'transformed_examples', 'updated_analyzer_cache'])


In [20]:
import pprint

import forest_fires_constants

_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_LABEL_KEY = forest_fires_constants.LABEL_KEY
pp = pprint.PrettyPrinter()

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.

example_records = []
float_features = [f+'_xf' for f in _DENSE_FLOAT_FEATURE_KEYS + ['area']]
int_features = [f+'_xf' for f in _BUCKET_FEATURE_KEYS + _VOCAB_FEATURE_KEYS]
for tfrecord in dataset.skip(200).take(5):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  record = [example.features.feature[f].int64_list.value for f in int_features] + [example.features.feature[f].float_list.value for f in float_features]
  example_records.append(record)
  print(example)
  print("="*50)

features {
  feature {
    key: "DC_xf"
    value {
      float_list {
        value: 0.9360822439193726
      }
    }
  }
  feature {
    key: "DMC_xf"
    value {
      float_list {
        value: 0.07424765080213547
      }
    }
  }
  feature {
    key: "FFMC_xf"
    value {
      float_list {
        value: 0.35225412249565125
      }
    }
  }
  feature {
    key: "ISI_xf"
    value {
      float_list {
        value: -0.3304247558116913
      }
    }
  }
  feature {
    key: "RH_xf"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "X_xf"
    value {
      float_list {
        value: -1.5708839893341064
      }
    }
  }
  feature {
    key: "Y_xf"
    value {
      float_list {
        value: -0.24281488358974457
      }
    }
  }
  feature {
    key: "area_xf"
    value {
      float_list {
        value: 7.210000038146973
      }
    }
  }
  feature {
    key: "day_xf"
    value {
      int64_list {
        value: 4
      }
    }
  }
  fea

In [21]:
sample_df = pd.DataFrame.from_records(example_records, columns=int_features+float_features)
sample_df.head()

,RH_xf,day_xf,month_xf,DC_xf,DMC_xf,FFMC_xf,ISI_xf,rain_xf,temp_xf,wind_xf,X_xf,Y_xf,area_xf
0,[0],[4],[1],[0.9360822439193726],[0.07424765080213547],[0.35225412249565125],[-0.3304247558116913],[-0.08785057067871094],[-0.4358100891113281],[0.03246145695447922],[-1.5708839893341064],[-0.24281488358974457],[7.210000038146973]
1,[0],[4],[1],[0.9360822439193726],[0.07424765080213547],[0.35225412249565125],[-0.3304247558116913],[-0.08785057067871094],[-0.4358100891113281],[0.03246145695447922],[0.5881509780883789],[0.539588987827301],[1.0099999904632568]
2,[1],[4],[1],[1.1011085510253906],[0.35142838954925537],[0.016011273488402367],[-0.4760842025279999],[-0.08785057067871094],[-1.1174758672714233],[-0.7055086493492126],[-0.7072699666023254],[0.539588987827301],[2.180000066757202]
3,[1],[4],[1],[0.6984107494354248],[-0.9431143999099731],[-0.4002879559993744],[-0.767403244972229],[-0.08785057067871094],[-0.9776469469070435],[-1.2164111137390137],[0.5881509780883789],[0.539588987827301],[4.420000076293945]
4,[0],[2],[1],[0.8017095923423767],[-0.18435117602348328],[0.25618404150009155],[-0.14314833283424377],[-0.08785057067871094],[0.8576073050498962],[-0.4784409999847412],[-1.5708839893341064],[-0.24281488358974457],[0.0]


In [33]:
#transform_output.transformed_feature_spec()
print(transform)

Transform(spec: <tfx.types.standard_component_specs.TransformSpec object at 0x00000159840DC048>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x00000159840DC0F0>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: Transform, inputs: {'examples': Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 1
type_id: 5
uri: "pipeline\\examples\\forest_fires_pipeline\\CsvExampleGen\\examples\\1"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:25478,xor_checksum:1623842899,sum_checksum:1623842899"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    string_value: "0"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }


In [60]:
# https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column
#print(transform.outputs["transformed_examples"])
a = tf.feature_column.numeric_column("a", shape=())
x = tf.keras.layers.DenseFeatures(a)({'a': [0.5, 0.6]})
print(x)

b = tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_identity('b', num_buckets=10))
y = tf.keras.layers.DenseFeatures(b)({'b': [5, 2]})
print(y)



tf.Tensor(
[[0.5]
 [0.6]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]], shape=(2, 10), dtype=float32)


"\nfor d in dataset.take(2):\n    serialized_example = d.numpy()\n    example = tf.train.Example()\n    example.ParseFromString(serialized_example)\n    print(example.features)\n    print(example.features.feature['RH_xf'].float_list.value)\n    x = tf.keras.layers.DenseFeatures(a)({'RH_xf': example.features.feature['RH_xf'].float_list})\n    print(x)"

In [ ]:
%%writefile forest_fires_trainer.py

import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils
from tfx_bsl.tfxio import dataset_options

import forest_fires_constants

_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_BUCKET_FEATURE_BOUNDARIES = forest_fires_constants.BUCKET_FEATURE_BOUNDARIES
_MAX_CATEGORICAL_FEATURE_VALUES = forest_fires_constants.MAX_CATEGORICAL_FEATURE_VALUES
_LABEL_KEY = forest_fires_constants.LABEL_KEY
_transformed_name = forest_fires_constants.transformed_name


def _transformed_names(keys):
  return [_transformed_name(key) for key in keys]


# Tf.Transform considers these features as "raw"
def _get_raw_feature_spec(schema):
  return schema_utils.schema_as_feature_spec(schema).feature_spec


def _build_estimator(config, hidden_units):
  """Build an estimator for predicting the tipping behavior of taxi riders.
  Args:
    config: tf.estimator.RunConfig defining the runtime environment for the
      estimator (including model_dir).
    hidden_units: [int], the layer sizes of the DNN (input layer first)
    warm_start_from: Optional directory to warm start from.
  Returns:
    A dict of the following:

      - estimator: The estimator that will be used for training and eval.
      - train_spec: Spec for training.
      - eval_spec: Spec for eval.
      - eval_input_receiver_fn: Input function for eval.
  """
  real_valued_columns = [
      tf.feature_column.numeric_column(key, shape=())
      for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  ]
  
  categorical_columns = [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=len(boundaries)+1
      ) for key, boundaries in zip(
          _transformed_names(_BUCKET_FEATURE_KEYS), _BUCKET_FEATURE_BOUNDARIES
      )
  ]
    
  categorical_columns += [
      tf.feature_column.categorical_column_with_identity(  # pylint: disable=g-complex-comprehension
          key,
          num_buckets=num_buckets,
          default_value=0) for key, num_buckets in zip(
              _transformed_names(_VOCAB_FEATURE_KEYS),
              _MAX_CATEGORICAL_FEATURE_VALUES)
  ]

  return tf.estimator.DNNLinearCombinedRegressor(
      config=config,
      linear_feature_columns=categorical_columns,
      dnn_feature_columns=real_valued_columns,
      dnn_hidden_units=hidden_units,
      dnn_activation_fn=tf.nn.leaky_relu,
      linear_optimizer=tf.keras.optimizers.RMSprop(lr=0.0001)
      )


def _example_serving_receiver_fn(tf_transform_graph, schema):
  """Build the serving in inputs.
  Args:
    tf_transform_graph: A TFTransformOutput.
    schema: the schema of the input data.
  Returns:
    Tensorflow graph which parses examples, applying tf-transform to them.
  """
  raw_feature_spec = _get_raw_feature_spec(schema)
  raw_feature_spec.pop(_LABEL_KEY)

  raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
      raw_feature_spec, default_batch_size=None)
  serving_input_receiver = raw_input_fn()

  transformed_features = tf_transform_graph.transform_raw_features(
      serving_input_receiver.features)

  return tf.estimator.export.ServingInputReceiver(
      transformed_features, serving_input_receiver.receiver_tensors)


def _eval_input_receiver_fn(tf_transform_graph, schema):
  """Build everything needed for the tf-model-analysis to run the model.
  Args:
    tf_transform_graph: A TFTransformOutput.
    schema: the schema of the input data.
  Returns:
    EvalInputReceiver function, which contains:

      - Tensorflow graph which parses raw untransformed features, applies the
        tf-transform preprocessing operators.
      - Set of raw, untransformed features.
      - Label against which predictions will be compared.
  """
  # Notice that the inputs are raw features, not transformed features here.
  raw_feature_spec = _get_raw_feature_spec(schema)

  serialized_tf_example = tf.compat.v1.placeholder(
      dtype=tf.string, shape=[None], name='input_example_tensor')

  # Add a parse_example operator to the tensorflow graph, which will parse
  # raw, untransformed, tf examples.
  features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)

  # Now that we have our raw examples, process them through the tf-transform
  # function computed during the preprocessing step.
  transformed_features = tf_transform_graph.transform_raw_features(
      features)

  # The key name MUST be 'examples'.
  receiver_tensors = {'examples': serialized_tf_example}

  # NOTE: Model is driven by transformed features (since training works on the
  # materialized output of TFT, but slicing will happen on raw features.
  features.update(transformed_features)

  return tfma.export.EvalInputReceiver(
      features=features,
      receiver_tensors=receiver_tensors,
      labels=transformed_features[_transformed_name(_LABEL_KEY)])


def _input_fn(file_pattern, data_accessor, tf_transform_output, batch_size=200):
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_transformed_name(_LABEL_KEY)),
      tf_transform_output.transformed_metadata.schema)


# TFX will call this function
def run_fn(trainer_fn_args, schema):
  """Build the estimator using the high level API.
  Args:
    trainer_fn_args: Holds args used to train the model as name/value pairs.
    schema: Holds the schema of the training examples.
  Returns:
    A dict of the following:

      - estimator: The estimator that will be used for training and eval.
      - train_spec: Spec for training.
      - eval_spec: Spec for eval.
      - eval_input_receiver_fn: Input function for eval.
  """
  
  train_batch_size = 50
  eval_batch_size = 50

  tf_transform_graph = tft.TFTransformOutput(trainer_fn_args.transform_output)

  train_input_fn = lambda: _input_fn(  # pylint: disable=g-long-lambda
      trainer_fn_args.train_files,
      trainer_fn_args.data_accessor,
      tf_transform_graph,
      batch_size=train_batch_size)

  eval_input_fn = lambda: _input_fn(  # pylint: disable=g-long-lambda
      trainer_fn_args.eval_files,
      trainer_fn_args.data_accessor,
      tf_transform_graph,
      batch_size=eval_batch_size)

  train_spec = tf.estimator.TrainSpec(  # pylint: disable=g-long-lambda
      train_input_fn,
      max_steps=trainer_fn_args.train_steps)

  serving_receiver_fn = lambda: _example_serving_receiver_fn(  # pylint: disable=g-long-lambda
      tf_transform_graph, schema)

  exporter = tf.estimator.FinalExporter('forest-fires', serving_receiver_fn)
  eval_spec = tf.estimator.EvalSpec(
      eval_input_fn,
      steps=trainer_fn_args.eval_steps,
      exporters=[exporter],
      name='forest-fires-eval')

  run_config = tf.estimator.RunConfig(
      save_checkpoints_steps=499, keep_checkpoint_max=1)

  run_config = run_config.replace(model_dir=trainer_fn_args.serving_model_dir)

  estimator = _build_estimator(
      # Construct layers sizes with exponetial decay
      hidden_units=[32],
      config=run_config)

  # Create an input receiver for TFMA processing
  receiver_fn = lambda: _eval_input_receiver_fn(  # pylint: disable=g-long-lambda
      tf_transform_graph, schema)

  return {
      'estimator': estimator,
      'train_spec': train_spec,
      'eval_spec': eval_spec,
      'eval_input_receiver_fn': receiver_fn
  }

In [78]:
%%writefile forest_fires_trainer.py

from typing import List, Text

import os
import absl
import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

import forest_fires_constants


_DENSE_FLOAT_FEATURE_KEYS = forest_fires_constants.DENSE_FLOAT_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = forest_fires_constants.VOCAB_FEATURE_KEYS
_BUCKET_FEATURE_KEYS = forest_fires_constants.BUCKET_FEATURE_KEYS
_BUCKET_FEATURE_BOUNDARIES = forest_fires_constants.BUCKET_FEATURE_BOUNDARIES
_MAX_CATEGORICAL_FEATURE_VALUES = forest_fires_constants.MAX_CATEGORICAL_FEATURE_VALUES
_LABEL_KEY = forest_fires_constants.LABEL_KEY
_transformed_name = forest_fires_constants.transformed_name


def _transformed_names(keys):
  return [_transformed_name(key) for key in keys]


def _get_serve_tf_examples_fn(model, tf_transform_output):
  """Returns a function that parses a serialized tf.Example and applies TFT."""

  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function
  def serve_tf_examples_fn(serialized_tf_examples):
    """Returns the output to be used in the serving signature."""
    feature_spec = tf_transform_output.raw_feature_spec()
    feature_spec.pop(_LABEL_KEY)
    parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
    transformed_features = model.tft_layer(parsed_features)
    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_transformed_name(_LABEL_KEY)),
      tf_transform_output.transformed_metadata.schema)


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying taxi data.

  Args:
    hidden_units: [int], the layer sizes of the DNN (input layer first).

  Returns:
    A keras Model.
  """
  real_valued_columns = [
      tf.feature_column.numeric_column(key, shape=())
      for key in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  ]
  
  categorical_columns = [
      tf.feature_column.indicator_column(
          tf.feature_column.categorical_column_with_identity(
              key, num_buckets=len(boundaries)+1
          )
      ) for key, boundaries in zip(
          _transformed_names(_BUCKET_FEATURE_KEYS), _BUCKET_FEATURE_BOUNDARIES
      )
  ]
    
  """categorical_columns += [
      tf.feature_column.indicator_column(
          tf.feature_column.categorical_column_with_identity(  # pylint: disable=g-complex-comprehension
              key,
              num_buckets=num_buckets,
              default_value=0
          ) for key, num_buckets in zip(
              _transformed_names(_VOCAB_FEATURE_KEYS),
              _MAX_CATEGORICAL_FEATURE_VALUES
          )
      )
  ]"""

  model = _dnn_regressor(
      # TODO(b/139668410) replace with premade wide_and_deep keras model
      columns=real_valued_columns+categorical_columns,
      dnn_hidden_units=[128, 64])
  return model


def _dnn_regressor(columns, dnn_hidden_units):
  """Build a simple keras wide and deep model.

  Args:
    wide_columns: Feature columns wrapped in indicator_column for wide (linear)
      part of the model.
    deep_columns: Feature columns for deep part of the model.
    dnn_hidden_units: [int], the layer sizes of the hidden DNN.

  Returns:
    A Wide and Deep Keras model
  """
  # Following values are hard coded for simplicity in this example,
  # However prefarably they should be passsed in as hparams.

  # Keras needs the feature definitions at compile time.
  # TODO(b/139081439): Automate generation of input layers from FeatureColumn.
  input_layers = {
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype=tf.float32)
      for colname in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS)
  }
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_VOCAB_FEATURE_KEYS)
  })
  input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_BUCKET_FEATURE_KEYS)
  })  

  # TODO(b/161952382): Replace with Keras preprocessing layers.
  output = tf.keras.layers.DenseFeatures(columns)(input_layers)
  for numnodes in dnn_hidden_units:
    output = tf.keras.layers.Dense(numnodes)(output)
  
  output = tf.keras.layers.Dense(1)(output)

  model = tf.keras.Model(input_layers, output)
  model.compile(
      loss='mean_squared_error',
      optimizer=tf.keras.optimizers.Adam(lr=0.001)
  )
  model.summary(print_fn=absl.logging.info)

  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, 40)
  eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, 40)

  model = _build_keras_model()
  
  os.makedirs(os.path.join('data','model_performance'), exist_ok=True)

  csv_callback = tf.keras.callbacks.CSVLogger(
    os.path.join('data', 'model_performance', 'model1.csv'), append=False
  )

  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      epochs=10,
      callbacks=[csv_callback]
  )

  signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model,
                                    tf_transform_output).get_concrete_function(
                                        tf.TensorSpec(
                                            shape=[None],
                                            dtype=tf.string,
                                            name='examples')),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)


Overwriting forest_fires_trainer.py


In [79]:
from tfx.components import Trainer
from tfx.proto import trainer_pb2
import tensorflow.keras.backend as K

K.clear_session()


trainer = Trainer(
    module_file=os.path.abspath("forest_fires_trainer.py"),
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=100))
context.run(trainer)


INFO:absl:Generating ephemeral wheel package for 'C:\\Users\\thush\\Documents\\code\\manning_tf2_in_action\\Ch14\\forest_fires_trainer.py' (including modules: ['forest_fires_constants', 'forest_fires_trainer', 'forest_fires_transform']).
INFO:absl:User module package has hash fingerprint version 550e9c8bef5a8b7a4f10ba22afdead638f73c8e73acbc0c24f56159f76f0646c.
INFO:absl:Executing: ['c:\\anaconda3\\envs\\manning.tf2\\python.exe', 'C:\\Users\\thush\\AppData\\Local\\Temp\\tmpdvxg7hbm\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\thush\\AppData\\Local\\Temp\\tmpcodqz29n', '--dist-dir', 'C:\\Users\\thush\\AppData\\Local\\Temp\\tmp1l_xzsqe']
INFO:absl:Successfully built user code wheel distribution at 'pipeline\\examples\\forest_fires_pipeline\\_wheels\\tfx_user_code_Trainer-0.0+550e9c8bef5a8b7a4f10ba22afdead638f73c8e73acbc0c24f56159f76f0646c-py3-none-any.whl'; target user module is 'forest_fires_trainer'.
INFO:absl:Full user module path is 'forest_fires_trainer@pipeli

INFO:absl:__________________________________________________________________________________________________
INFO:absl:month_xf (InputLayer)           [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:rain_xf (InputLayer)            [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:temp_xf (InputLayer)            [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:wind_xf (InputLayer)            [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:dense_fea

Epoch 1/10
 95/100 [===========================>..] - ETA: 53s - loss: 315.86 - ETA: 0s - loss: 2025.46 - ETA: 0s - loss: 3540.41 - ETA: 0s - loss: 4567.42 - ETA: 0s - loss: 4913.73 - ETA: 0s - loss: 5092.15 - ETA: 0s - loss: 5213.12 - ETA: 0s - loss: 5333.57 - ETA: 0s - loss: 5402.81 - ETA: 0s - loss: 5432.22 - ETA: 0s - loss: 5469.18 - ETA: 0s - loss: 5505.41 - ETA: 0s - loss: 5525.83 - ETA: 0s - loss: 5538.72 - ETA: 0s - loss: 5542.2316WARNING:tensorflow:11 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x0000016D363D80D0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessa

Exception ignored in: <bound method CapturableResourceDeleter.__del__ of <tensorflow.python.training.tracking.tracking.CapturableResourceDeleter object at 0x0000016D4AE23080>>
Traceback (most recent call last):
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\training\tracking\tracking.py", line 208, in __del__
    self._destroy_resource()
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\def_function.py", line 726, in _initialize
    *args, **kwds))
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\function.py", line 2969, in _get_concrete_function_internal_garbage

    *args, **kwds))
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\function.py", line 2969, in _get_concrete_function_internal_garbage_collected
    graph_function, _ = self._maybe_define_function(args, kwargs)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\function.py", line 3361, in _maybe_define_function
    graph_function = self._create_graph_function(args, kwargs)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\function.py", line 3206, in _create_graph_function
    capture_by_value=self._capture_by_value),
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\framework\func_graph.py", line 990, in func_graph_from_py_func
    func_outputs = python_func(*func_args, **func_kwargs)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\def_function.py", line 634, in wrapped_fn
    out = weak_wrapped_fn().__wrapped__(*args, **kwds)
  File "c:\an

    cancellation_manager)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\function.py", line 1932, in _call_flat
    flat_outputs = forward_function.call(ctx, args_with_tangents)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\function.py", line 589, in call
    executor_type=executor_type)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\ops\functional_ops.py", line 1206, in partitioned_call
    f.add_to_graph(graph)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\function.py", line 505, in add_to_graph
    g._add_function(self)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\framework\ops.py", line 3396, in _add_function
    gradient)
tensorflow.python.framework.errors_impl.InvalidArgumentError: 'func' argument to TF_GraphCopyFunction cannot be null
Exception ignored in: <bound method CapturableResourceDeleter.__del__ of <tensorflow.python

100/100 [==============================] - 4s 32ms/step - loss: 5566.0167 - val_loss: 764.6214
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 907.200 - ETA: 0s - loss: 786.638 - ETA: 0s - loss: 2131.33 - ETA: 0s - loss: 2539.52 - ETA: 0s - loss: 3111.16 - ETA: 0s - loss: 3418.89 - ETA: 0s - loss: 3652.22 - ETA: 0s - loss: 3820.67 - ETA: 0s - loss: 3931.61 - ETA: 0s - loss: 4040.37 - ETA: 0s - loss: 4143.64 - ETA: 0s - loss: 4238.16 - ETA: 0s - loss: 4315.19 - ETA: 0s - loss: 4377.88 - ETA: 0s - loss: 4431.01 - 1s 14ms/step - loss: 4475.8067 - val_loss: 817.8895
Epoch 3/10
100/100 [==============================] - ETA: 0s - loss: 13372.581 - ETA: 0s - loss: 6525.778 - ETA: 0s - loss: 5778.36 - ETA: 0s - loss: 5610.75 - ETA: 0s - loss: 5667.19 - ETA: 0s - loss: 5708.52 - ETA: 0s - loss: 5644.55 - ETA: 0s - loss: 5656.44 - ETA: 0s - loss: 5650.47 - ETA: 0s - loss: 5608.31 - ETA: 0s - loss: 5612.39 - ETA: 0s - loss: 5611.03 - ETA: 0s - loss: 5598.85 - ETA: 0s - loss

INFO:tensorflow:Assets written to: pipeline\examples\forest_fires_pipeline\Trainer\model\58\Format-Serving\assets


INFO:tensorflow:Assets written to: pipeline\examples\forest_fires_pipeline\Trainer\model\58\Format-Serving\assets
INFO:absl:Training complete. Model written to pipeline\examples\forest_fires_pipeline\Trainer\model\58\Format-Serving. ModelRun written to pipeline\examples\forest_fires_pipeline\Trainer\model_run\58
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 58
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 70
        type_id: 14
        uri: "pipeline\\examples\\forest_fires_pipeline\\Trainer\\model\\58"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 71
        type_id: 15
        uri: "pipeline\\examples\\forest_fires_pipeline\\Trainer\\model_run\\58"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 15
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
## Remove anomalies

In [80]:
df = pd.read_csv(os.path.join('data','csv','forestfires.csv'), header=0)
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [81]:
print("Original shape: {}".format(df.shape))
valid_df = df[(df["FFMC"]>=60)&(df["ISI"]<=30)&(df["Y"]<=7)&(df["area"]<=75)&(df["rain"]<=1)]
print("After anomalies removed shape: {}".format(valid_df.shape))
os.makedirs(os.path.join('data','csv_valid'),exist_ok=True)
valid_df.to_csv(os.path.join('data','csv_valid','forestfires.csv'), header=True, index=False)

Original shape: (517, 13)
After anomalies removed shape: (490, 13)


In [82]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=3),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))


example_gen = CsvExampleGen(input_base=os.path.join('data', 'csv_valid'))
context.run(example_gen)

from tfx.components import StatisticsGen

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

context.show(statistics_gen.outputs['statistics'])

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:Generating examples.
INFO:absl:Processing input csv data data\csv_valid\* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to pipeline\examples\forest_fires_pipeline\StatisticsGen\statistics\60\Split-train.
INFO:abs

In [83]:
from tfx.components import Transform

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath('forest_fires_transform.py'))

context.run(transform)

INFO:absl:Generating ephemeral wheel package for 'C:\\Users\\thush\\Documents\\code\\manning_tf2_in_action\\Ch14\\forest_fires_transform.py' (including modules: ['forest_fires_constants', 'forest_fires_trainer', 'forest_fires_transform']).
INFO:absl:User module package has hash fingerprint version 550e9c8bef5a8b7a4f10ba22afdead638f73c8e73acbc0c24f56159f76f0646c.
INFO:absl:Executing: ['c:\\anaconda3\\envs\\manning.tf2\\python.exe', 'C:\\Users\\thush\\AppData\\Local\\Temp\\tmp_9hdfkw7\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\thush\\AppData\\Local\\Temp\\tmpiqcpnr69', '--dist-dir', 'C:\\Users\\thush\\AppData\\Local\\Temp\\tmpeyc2syr6']
INFO:absl:Successfully built user code wheel distribution at 'pipeline\\examples\\forest_fires_pipeline\\_wheels\\tfx_user_code_Transform-0.0+550e9c8bef5a8b7a4f10ba22afdead638f73c8e73acbc0c24f56159f76f0646c-py3-none-any.whl'; target user module is 'forest_fires_transform'.
INFO:absl:Full user module path is 'forest_fires_transfor

INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature FFMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ISI has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature RH has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature X has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Y has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature rain has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature wind has no shape. Setting to VarLenSparseTensor.


INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature FFMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ISI has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature RH has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature X has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Y has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature rain has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature wind has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparse

INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature FFMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ISI has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature RH has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature X has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Y has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature rain has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature wind has no shape. Setting to VarLenSparseTensor.


INFO:absl:Feature day has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature month has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature FFMC has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature ISI has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature RH has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature X has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Y has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature area has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature rain has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature wind has no shape. Setting to VarLenSparseTensor.


...

INFO:tensorflow:Assets written to: pipeline\examples\forest_fires_pipeline\Transform\transform_graph\61\.temp_path\tftransform_tmp\259f14d8092941949a1de38c48fab899\assets


INFO:tensorflow:Assets written to: pipeline\examples\forest_fires_pipeline\Transform\transform_graph\61\.temp_path\tftransform_tmp\3aa553bd80f94129ac951d8b89004ca5\assets


INFO:tensorflow:Assets written to: pipeline\examples\forest_fires_pipeline\Transform\transform_graph\61\.temp_path\tftransform_tmp\3aa553bd80f94129ac951d8b89004ca5\assets


INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 61
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 74
        type_id: 11
        uri: "pipeline\\examples\\forest_fires_pipeline\\Transform\\transform_graph\\61"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 75
        type_id: 5
        uri: "pipeline\\examples\\forest_fires_pipeline\\Transform\\transformed_examples\\61"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 76
        type_id: 12
        uri: "pipeline\\examples\\forest_fires_pipeline\\Transform\\updated_analyzer_cache\\61"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 12
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [84]:
from tfx.components import Trainer
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=os.path.abspath("forest_fires_trainer.py"),
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=100))
context.run(trainer)


INFO:absl:Generating ephemeral wheel package for 'C:\\Users\\thush\\Documents\\code\\manning_tf2_in_action\\Ch14\\forest_fires_trainer.py' (including modules: ['forest_fires_constants', 'forest_fires_trainer', 'forest_fires_transform']).
INFO:absl:User module package has hash fingerprint version 550e9c8bef5a8b7a4f10ba22afdead638f73c8e73acbc0c24f56159f76f0646c.
INFO:absl:Executing: ['c:\\anaconda3\\envs\\manning.tf2\\python.exe', 'C:\\Users\\thush\\AppData\\Local\\Temp\\tmpej1tr8a_\\_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', 'C:\\Users\\thush\\AppData\\Local\\Temp\\tmp8g5in67n', '--dist-dir', 'C:\\Users\\thush\\AppData\\Local\\Temp\\tmpg5w0h91w']
INFO:absl:Successfully built user code wheel distribution at 'pipeline\\examples\\forest_fires_pipeline\\_wheels\\tfx_user_code_Trainer-0.0+550e9c8bef5a8b7a4f10ba22afdead638f73c8e73acbc0c24f56159f76f0646c-py3-none-any.whl'; target user module is 'forest_fires_trainer'.
INFO:absl:Full user module path is 'forest_fires_trainer@pipeli

INFO:absl:__________________________________________________________________________________________________
INFO:absl:month_xf (InputLayer)           [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:rain_xf (InputLayer)            [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:temp_xf (InputLayer)            [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:wind_xf (InputLayer)            [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:dense_fea

Epoch 1/10
100/100 [==============================] - ETA: 47s - loss: 121.36 - ETA: 0s - loss: 155.7430 - ETA: 0s - loss: 166.852 - ETA: 0s - loss: 169.169 - ETA: 0s - loss: 166.891 - ETA: 0s - loss: 165.509 - ETA: 0s - loss: 163.602 - ETA: 0s - loss: 162.212 - ETA: 0s - loss: 161.249 - ETA: 0s - loss: 160.354 - ETA: 0s - loss: 159.336 - ETA: 0s - loss: 158.767 - ETA: 0s - loss: 158.113 - ETA: 0s - loss: 157.221 - ETA: 0s - loss: 156.432 - ETA: 0s - loss: 155.724 - 2s 17ms/step - loss: 155.6339 - val_loss: 146.6859
Epoch 2/10
100/100 [==============================] - ETA: 0s - loss: 131.364 - ETA: 0s - loss: 127.976 - ETA: 0s - loss: 130.697 - ETA: 0s - loss: 131.728 - ETA: 0s - loss: 133.186 - ETA: 0s - loss: 133.828 - ETA: 0s - loss: 134.258 - ETA: 0s - loss: 134.954 - ETA: 0s - loss: 135.489 - ETA: 0s - loss: 135.600 - ETA: 0s - loss: 135.724 - ETA: 0s - loss: 135.890 - ETA: 0s - loss: 136.072 - ETA: 0s - loss: 136.234 - ETA: 0s - loss: 136.246 - 1s 15ms/step - loss: 136.3256 - va

Exception ignored in: <bound method CapturableResourceDeleter.__del__ of <tensorflow.python.training.tracking.tracking.CapturableResourceDeleter object at 0x0000016D3DB1C940>>
Traceback (most recent call last):
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\training\tracking\tracking.py", line 208, in __del__
    self._destroy_resource()
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\def_function.py", line 871, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\def_function.py", line 726, in _initialize
    *args, **kwds))
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\function.py", line 2969, in _get_concrete_function_internal_garbage

    *args, **kwds))
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\function.py", line 2969, in _get_concrete_function_internal_garbage_collected
    graph_function, _ = self._maybe_define_function(args, kwargs)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\function.py", line 3361, in _maybe_define_function
    graph_function = self._create_graph_function(args, kwargs)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\function.py", line 3206, in _create_graph_function
    capture_by_value=self._capture_by_value),
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\framework\func_graph.py", line 990, in func_graph_from_py_func
    func_outputs = python_func(*func_args, **func_kwargs)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\def_function.py", line 634, in wrapped_fn
    out = weak_wrapped_fn().__wrapped__(*args, **kwds)
  File "c:\an

    cancellation_manager)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\function.py", line 1932, in _call_flat
    flat_outputs = forward_function.call(ctx, args_with_tangents)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\function.py", line 589, in call
    executor_type=executor_type)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\ops\functional_ops.py", line 1206, in partitioned_call
    f.add_to_graph(graph)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\eager\function.py", line 505, in add_to_graph
    g._add_function(self)
  File "c:\anaconda3\envs\manning.tf2\lib\site-packages\tensorflow\python\framework\ops.py", line 3396, in _add_function
    gradient)
tensorflow.python.framework.errors_impl.InvalidArgumentError: 'func' argument to TF_GraphCopyFunction cannot be null
Exception ignored in: <bound method CapturableResourceDeleter.__del__ of <tensorflow.python

100/100 [==============================] - 3s 27ms/step - loss: 135.8612 - val_loss: 146.8513
Epoch 5/10
100/100 [==============================] - ETA: 0s - loss: 148.814 - ETA: 0s - loss: 132.966 - ETA: 0s - loss: 133.980 - ETA: 0s - loss: 135.166 - ETA: 0s - loss: 135.461 - ETA: 0s - loss: 136.461 - ETA: 0s - loss: 136.750 - ETA: 0s - loss: 137.002 - ETA: 0s - loss: 136.992 - ETA: 0s - loss: 137.188 - ETA: 0s - loss: 137.316 - ETA: 0s - loss: 137.394 - ETA: 0s - loss: 137.311 - ETA: 0s - loss: 137.389 - ETA: 0s - loss: 137.469 - ETA: 0s - loss: 137.473 - ETA: 0s - loss: 137.450 - 1s 15ms/step - loss: 137.4836 - val_loss: 147.7143
Epoch 6/10
100/100 [==============================] - ETA: 1s - loss: 172.922 - ETA: 0s - loss: 147.358 - ETA: 0s - loss: 137.752 - ETA: 0s - loss: 137.133 - ETA: 0s - loss: 136.308 - ETA: 0s - loss: 136.339 - ETA: 0s - loss: 136.213 - ETA: 0s - loss: 136.153 - ETA: 0s - loss: 136.207 - ETA: 0s - loss: 136.091 - ETA: 0s - loss: 135.973 - ETA: 0s - loss: 136

INFO:tensorflow:Assets written to: pipeline\examples\forest_fires_pipeline\Trainer\model\62\Format-Serving\assets


INFO:tensorflow:Assets written to: pipeline\examples\forest_fires_pipeline\Trainer\model\62\Format-Serving\assets
INFO:absl:Training complete. Model written to pipeline\examples\forest_fires_pipeline\Trainer\model\62\Format-Serving. ModelRun written to pipeline\examples\forest_fires_pipeline\Trainer\model_run\62
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 62
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 77
        type_id: 14
        uri: "pipeline\\examples\\forest_fires_pipeline\\Trainer\\model\\62"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 78
        type_id: 15
        uri: "pipeline\\examples\\forest_fires_pipeline\\Trainer\\model_run\\62"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 15
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))